In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import pickle
import json
import requests
import numpy as np
from scipy.stats import truncnorm

In [2]:
print(pickle.format_version)

4.0


### 2. POST request on the `performance` endpoint for both regressor and classifier

In [3]:
class_test_file = open('classification_test.json')
class_test_json = json.load(class_test_file)

regr_test_file = open('regression_test.json')
regr_test_json = json.load(regr_test_file)

In [4]:
print(type(class_test_json))
print(type(regr_test_json))

<class 'dict'>
<class 'dict'>


#### 2.1 POST done passing `classifier` as parameter to the endpoint.

In [5]:
home = 'http://localhost:8080'
classifier_endpoint = '/performance/classifier'
headers = {'Content-Type': 'application/json', 'accept': 'application/json'}

# POST done at the /performance/{model_from} endpoint
res_classifier = requests.post(home + classifier_endpoint, json=class_test_json, headers=headers)

The answer is returned...

This latency stands for the time required for the model to perform prediction over the data.

In [6]:
res_classifier_parsed = json.loads(res_classifier.text)
print(json.dumps(res_classifier_parsed, indent=4))

{
    "evaluation_metric": {
        "f1": "0.8936170212765957"
    },
    "satisficing_metrics": {
        "precision": "0.9130434782608695",
        "recall": "0.875",
        "latency_ms": "2.0279884338378906"
    }
}


#### 2.2 POST done passing `regressor` as parameter to the endpoint.

In [7]:
regressor_endpoint = '/performance/regressor'

# POST done at the /performance/{model_from} endpoint
res_regressor = requests.post(home + regressor_endpoint, json=regr_test_json, headers=headers)

The answer is returned...

In [8]:
res_regressor_parsed = json.loads(res_regressor.text)
print(json.dumps(res_regressor_parsed, indent=4))

{
    "evaluation_metric": {
        "RMSE": "8.018073509187136"
    },
    "satisficing_metrics": {
        "R2": "0.9999597127306887",
        "correlation": {
            "statistic": "0.9999882061079758",
            "p-value": "0.0"
        },
        "latency_ms": "1.0030269622802734"
    }
}


### 3. POST request on the `adherence` endpoint only for classifier

The idea to create this endpoint is to enable data drifting monitoring. This endpoint performs the Kolmogorov-Smirnov test considering two sample distributions in order to determine the distance between them. Depending on a predefined significance level, it is possible to say whether these distributions come from the same population. When they do not, it is an indicative that the data collected is slowly drifting and <strong>should</strong> harm model performance. It is useful to monitor it to prevent these situations.

Let this significance level be $\alpha = 0.05$

To get to some conclusion, let a null hypothesis consisting in saying both samples <strong>belong</strong> to the same distribution.

This way, using <em><strong>scipy.stats.ks_2samp(score_req, score_test)</strong></em> we consider as null hypothesis the fact that the probability scores determined by the classifier model to <strong>both</strong> datasets do come from the same distribution. This implies, hence, that these data were brought by the same population.

Therefore, by calculating the KS test metrics, we shall compare <em><strong>p-value</strong></em> with <em><strong>$\alpha$</strong></em> to either reject or not the null hypothesis

<li>If <em><strong>p-value</strong></em> $ > \alpha$, we do not reject the null hypothesis and the samples <strong>do</strong> come from the same distribution of data</li>
<li>If <em><strong>p-value</strong></em> $ <= \alpha$, we reject the null hypothesis and the samples <strong>do not</strong> come from the same distribution of data</li>

The KS test uses the <em><strong>statistic</strong></em> metric to define the <em><strong>p-value</strong></em>.

#### 3.1 Get reference data

The fitted model expects to predict over datasets that share probabilistic properties with the reference data (the one the model was trained on).

In [9]:
class_train_file = open('classification_train.json')
class_train_json = json.load(class_train_file)

#### 3.2 Get predefined testing data and perform POST passing `classifier` as parameter to the endpoint.

In [10]:
type(class_test_json)

dict

In [11]:
json_data = {'reference': class_train_json, 'request': class_test_json}

In [12]:
classifier_endpoint_adh = '/adherence/classifier'

# POST done at the /adherence/{model_from} endpoint
res_classifier_adh = requests.post(home + classifier_endpoint_adh, json=json_data, headers=headers)

The answer is returned...

The sample datasets do come from the same distribution of data as `p-value` is indeed greater than `.05`.

In [13]:
res_classifier_adh_parsed = json.loads(res_classifier_adh.text)
print(json.dumps(res_classifier_adh_parsed, indent=4))

{
    "KStest-result": {
        "statistic": 0.04690185436454093,
        "p-value": 0.6924656909900425
    }
}


This endpoint could be automated in order to perform this test in a daily basis so that it could trigger alarms whenever there are signs of data drifting as the `p-value` gets lower than the significance level `0.05`.